# Convert labelme output (json) to yolov5 input (txt)
Author: Yixing
Date: 2021-07-19 18:17:53
Software: VS Code

In [1]:
import os
import numpy as np
import json
from glob import glob
import cv2
from sklearn.model_selection import train_test_split
from os import getcwd
import shutil
 
classes = ['eraser']

In [3]:
# the dir of labels
labelme_path = "dataset\\annotation_yolo\\"
raw_img_path = 'dataset\\raw imgs\\'

isUseTest = False  # 是否创建test集

# get all json label files
files = glob(labelme_path + "*.json")
files = [i.replace("\\", "/").split("/")[-1].split(".json")[0] for i in files]
len(files)

51

In [11]:
# split train/val and test set
if isUseTest:
    trainval_files, test_files = train_test_split(files, test_size=0.1, random_state=55)
else:
    trainval_files = files

In [12]:
# split train and val set
train_files, val_files = train_test_split(trainval_files, test_size=0.1, random_state=55)

In [13]:
def convert(size, box):
    dw = 1. / (size[0])
    dh = 1. / (size[1])
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

In [14]:
wd = getcwd()
print(wd)

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn


In [17]:
def ChangeToYolo5(files, txt_Name):
    if not os.path.exists('tmp/'):
        os.makedirs('tmp/')
    list_file = open('tmp/%s.txt' % (txt_Name), 'w') # open the dir and create txt files
    for json_file_ in files:
        json_filename = labelme_path + json_file_ + ".json" # json file path
        imagePath = raw_img_path + json_file_ + ".jpg" # raw image file path
        list_file.write('%s\\%s\n' % (wd, imagePath))
        out_file = open('%s\\%s.txt' % (labelme_path, json_file_), 'w') # path to txt (create txt)
        json_file = json.load(open(json_filename, "r", encoding="utf-8"))
        height, width, channels = cv2.imread(raw_img_path + json_file_ + ".jpg").shape
        for multi in json_file["shapes"]:
            points = np.array(multi["points"])
            xmin = min(points[:, 0]) if min(points[:, 0]) > 0 else 0
            xmax = max(points[:, 0]) if max(points[:, 0]) > 0 else 0
            ymin = min(points[:, 1]) if min(points[:, 1]) > 0 else 0
            ymax = max(points[:, 1]) if max(points[:, 1]) > 0 else 0
            label = multi["label"]
            if xmax <= xmin:
                pass
            elif ymax <= ymin:
                pass
            else:
                cls_id = classes.index(label)
                b = (float(xmin), float(xmax), float(ymin), float(ymax))
                bb = convert((width, height), b)
                out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
                print(json_filename, xmin, ymin, xmax, ymax, cls_id)

In [18]:
ChangeToYolo5(train_files, "train")
ChangeToYolo5(val_files, "val")
ChangeToYolo5(test_files, "test")

dataset\annotation_yolo\00026.json 49.25 202.375 938.625 748.0 0
dataset\annotation_yolo\00011.json 87.75700934579436 214.9532710280374 1443.8317757009345 1181.3084112149531 0
dataset\annotation_yolo\00005.json 312.51282051282055 754.2564102564103 1733.025641025641 1294.0 0
dataset\annotation_yolo\00034.json 31.568345323741 12.949640287769778 180.69475847893113 81.80883864337102 0
dataset\annotation_yolo\00034.json 31.362795477903404 80.06166495375128 181.20863309352518 148.5097636176773 0
dataset\annotation_yolo\00029.json 24.0 214.5 231.5 447.0 0
dataset\annotation_yolo\00029.json 275.0 189.0 478.0 408.5 0
dataset\annotation_yolo\00029.json 502.0 254.5 740.5 442.0 0
dataset\annotation_yolo\00029.json 490.0 475.0 720.0 650.5 0
dataset\annotation_yolo\00032.json 51.450980392156865 101.08932461873638 296.76688453159045 256.2091503267974 0
dataset\annotation_yolo\00019.json 331.25 676.0 1662.5 1172.25 0
dataset\annotation_yolo\00012.json 96.16822429906534 346.7289719626168 1386.822429906

## make dataset

In [25]:
# make dataset
file_List = ["train", "val", "test"]
for file in file_List:
    # make dirs if not exist
    if not os.path.exists('VOC/images/%s' % file):
        os.makedirs('VOC/images/%s' % file)
    if not os.path.exists('VOC/labels/%s' % file):
        os.makedirs('VOC/labels/%s' % file)
    # 判断tmp里面是否存在train, val, test三个文件夹
    # 输出->True：存在  False：不存在
    print(os.path.exists('tmp/%s.txt' % file))
    # 打开tmp中train, val和test
    f = open('tmp/%s.txt' % file, 'r')
    # 读取每个文件夹里面存放的图片地址
    lines = f.readlines()
    for line in lines:
        print(line)
        # Python join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串
        line = "/".join(line.split('\\')[-3:]).strip()
        shutil.copy(line, "VOC/images/%s" % file)
        label_path = 'dataset\\annotation_yolo'.replace('\\', '/')
        label_fname = line.split('/')[-1].replace('jpg', 'txt').strip()
        line = label_path + '/' + label_fname
        shutil.copy(line, "VOC/labels/%s/" % file)

True
c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00026.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00011.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00005.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00034.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00029.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00032.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00019.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00012.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00041.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00040.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00016.jpg

c:\Users\Luyixing\Documents\2021intern\proj_mask_rcnn\dataset\raw imgs\00018.jpg

c:\Users\Lu

## dataset info

In [27]:
train_set = r'VOC\images\train'
val_set = r'VOC\images\val'
test_set = r'VOC\images\test'

dataset_list = [train_set, val_set, test_set]

for dset in dataset_list:
    print(str(dset), ':' , len(os.listdir(dset)))

VOC\images\train : 40
VOC\images\val : 5
VOC\images\test : 6
